In [8]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'config.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from apps.gym import models
from apps.gym.selectors import (
    ExercisesSelector, 
    ListExercisesLookups,
    MuscleGroupsSelector,
)
from apps.core.utils import BaseLookupDTO
from apps.core.constants import Language
from django.db.models import Prefetch

from django.db import connection
from django.db.models import F
from django.contrib.postgres.fields import jsonb
from django.http import JsonResponse


In [10]:
muscle = models.MuscleGroups.objects.all()[0]
muscle

<MuscleGroups (name='{'en': 'abdominals', 'pt': 'abdominais'}')>

In [15]:
qs = ExercisesSelector.list_exercises(
    language=Language.PT,
    lookups=ListExercisesLookups(
        primary_muscle=muscle,
    )
)
qs

[{'id': UUID('42862e97-a527-4344-8d58-75024d5a76fc'),
  'name': 'Ab Scissors',
  'muscle': 'abdominais'},
 {'id': UUID('909c5904-f82a-4275-bb67-e1820fce7361'),
  'name': 'Bicycle Crunch',
  'muscle': 'abdominais'},
 {'id': UUID('2768cf93-eebe-4266-a4be-a0f6fa7b9892'),
  'name': 'Cable Crunch',
  'muscle': 'abdominais'},
 {'id': UUID('befc74ff-30cc-41a1-9a56-c6d4d9cae552'),
  'name': 'Cable Twist',
  'muscle': 'abdominais'},
 {'id': UUID('6a162bcc-b678-4275-b40f-cf491a9e3775'),
  'name': 'Crunch',
  'muscle': 'abdominais'},
 {'id': UUID('707ba5ea-c88b-487d-8464-f184d6e7ef84'),
  'name': 'Crunch',
  'muscle': 'abdominais'},
 {'id': UUID('8ad77a98-2e63-403c-8b57-47a3d3e1556a'),
  'name': 'Crunch',
  'muscle': 'abdominais'},
 {'id': UUID('2484d226-cdee-4121-a7ae-24ff92d5ef6f'),
  'name': 'Dead Bug',
  'muscle': 'abdominais'},
 {'id': UUID('25ab0bcc-05a6-4cbd-831f-ef1878e8f942'),
  'name': 'Decline Crunch',
  'muscle': 'abdominais'},
 {'id': UUID('a1b84d30-7870-49ed-9d72-07b480ad1dab'),
  '

In [14]:
muscles_qs = MuscleGroupsSelector.list_muscles(language=Language.PT)
muscles_qs

[{'id': UUID('a048f875-4732-41c5-8eef-8a9dc313a19f'), 'name': 'abdominais'},
 {'id': UUID('a7723984-6af0-48c7-a5ae-8acfe529b441'), 'name': 'abdutores'},
 {'id': UUID('211383da-0b5b-494d-9242-b8924282c9df'), 'name': 'adutores'},
 {'id': UUID('47413ba9-056f-4057-9d1f-b5fcf05213f6'), 'name': 'pescoço'},
 {'id': UUID('a25eec4e-7953-4642-9711-3b35e7c556dc'), 'name': 'bíceps'},
 {'id': UUID('6a4d7fb5-9182-4010-8ec3-12bd0668b476'),
  'name': 'parte superior das costas'},
 {'id': UUID('800aae8b-06cc-4d4f-b5be-078abdef201d'), 'name': 'cardio'},
 {'id': UUID('093dcd25-7090-4e51-9744-2484874c7053'), 'name': 'panturrilhas'},
 {'id': UUID('962f919e-bfa7-4254-9d61-2885b6f07641'), 'name': 'glúteos'},
 {'id': UUID('36ca3685-e055-4a6d-a2cf-2994b0f6c887'), 'name': 'lombares'},
 {'id': UUID('7f2898c4-cfe0-4dad-b135-5025ec17db0f'), 'name': 'quadríceps'},
 {'id': UUID('77f62268-13dd-4d33-a0b9-8eeec6b10a7c'), 'name': 'tríceps'},
 {'id': UUID('6372da93-5665-41b1-ad50-24f505432a06'), 'name': 'corpo inteiro'},

In [ ]:
models.Exercises.objects.annotate(
    name=F('name__pt')
).values('name')

In [ ]:
qs = models.Exercises.objects.filter(name__pt='Supino')
for i in qs:
    print(i.name)
    print(i.pk)

In [ ]:
qs = models.Exercises.objects.values('id').annotate(
    name=F('name__pt')
).values('name')
qs = qs.filter(name='Supino')
# qs.filter(name=None)
qs

In [ ]:
connection.queries

In [ ]:
from django.db.models import F, CharField
from django.db.models.expressions import ExpressionWrapper
from django.db.models.functions import Cast, JSONObject

models.Exercises.objects.annotate(
    portuguese_name=Cast(
        F('translations__pt__name'),
        output_field=CharField()
    )
).values('id', 'portuguese_name')[0]['portuguese_name']


In [ ]:
qs = ExercisesSelector.list_exercises()
qs

In [ ]:
list(qs)

In [ ]:
JsonResponse(dict(a=list(qs)))

In [ ]:
class TestLookup(BaseLookupDTO):
    some: str
    value: str

In [ ]:
a = TestLookup(
    some=1,
    value=2,
)